In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.font_manager

In [3]:
pip install pyod

Processing c:\users\nitin\appdata\local\pip\cache\wheels\ea\11\60\e16348f27a3514f257e143d51e38ebc513eaec4bad0261cd12\pyod-2.0.2-py3-none-any.whl
  Using cached scipy-1.7.3-cp37-cp37m-win_amd64.whl (34.1 MB)
  Using cached numba-0.56.4-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached llvmlite-0.39.1-cp37-cp37m-win_amd64.whl (23.2 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.31.0


ERROR: statsmodels 0.13.5 has requirement packaging>=21.3, but you'll have packaging 20.1 which is incompatible.
ERROR: sentence-transformers 2.2.2 has requirement torch>=1.6.0, but you'll have torch 1.3.1+cpu which is incompatible.
ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
from pyod.models.knn import KNN
from pyod.utils.data import generate_data, get_outliers_inliers

ModuleNotFoundError: No module named 'pyod'

In [ ]:
x_tr, y_tr = generate_data(n_train = 300, train_only = True, n_features = 2)

outlier_fraction = 0.1

x_outliers, x_inliers= get_outliers_inliers(x_tr, y_tr)
n_inliers = len(x_inliers)
n_outliers = len(x_outliers)

#Separating two features
f1 = x_tr[:, [0]].reshape(-1,1)
f2 = x_tr[:,[1]].reshape(-1,1)

In [ ]:
#Creating scatter plot
x,y = np.meshgrid(np.linspace(-10,10,200), np.linspace(-10,10,200))

plt.scatter(f1,f2)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

In [ ]:
cls = KNN(contamination = outlier_fraction)
cls.fit(x_tr, y_tr)

pred = cls.decision_function(x_tr)*-1

y_pred = cls.predict(x_tr)
n_errors = (y_pred != y_tr).sum()

In [ ]:
#Threshhold value to decide inlier or outliers
threshold = stats.scoreatpercentile(pred, 100*outlier_fraction)

#Calculating raw anomaly score for each point using Decision function
z = cls.decision_function(np.c_[x.ravel(), y.ravel()])*-1
z = z.reshape(x.shape)

subplot = plt.subplot(1,2,1)
subplot.contourf(x,y,z,levels = np.linspace(z.min(), threshold, 10), cmap = plt.cm.Blues_r)

#Drawing Decision boundry based on anomaly score
a = subplot.contour(x,y,z,levels = [threshold], linewidths = 2, colors='red')

subplot.contourf(x,y,z, levels = [threshold, z.max()], colors = 'Orange')

b = subplot.scatter(x_tr[ :-n_outliers, 0], x_tr[ :-n_outliers,1 ], c = 'White',
                   s = 20, edgecolor = 'k')

c = subplot.scatter(x_tr[-n_outliers: , 0], x_tr[-n_outliers: ,1 ], c = 'Black',
                   s = 20, edgecolor = 'k')

subplot.legend([a.collections[0],b,c],
              ['learned decision functions', 'Inliers', 'Outliers'],
              prop = matplotlib.font_manager.FontProperties(size = 10),
              loc = 'lower right')

subplot.set_title('Outliers Detection Using KNN')
subplot.set_xlim((-10,10))
subplot.set_ylim((-10,10))
plt.show()
